# Custom Container Serving AI-Platform (uCAIP)
Steps for creating and serving prediction from a custom container

In [5]:
#Replce the following values with your own
PROJECT_ID = "[PROJECT-ID-GOES-HERE]"
REGION = "us-central1"
ARTIFACT_REPO = "repo-flask"
CONTAINER = "container_flask"
MODEL_NAME = "model_python"
MODEL_VERSION = "v4"

## Setup access to GCP project

In [ ]:
!gcloud auth login
#Might need to run in a terminal to click followup link

Configure access to the repo
https://cloud.google.com/artifact-registry/docs/docker/authentication#gcloud-helper

## Docker

In [2]:
TAG = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPO}/{CONTAINER}:latest"
print(TAG)
!docker build -t $TAG .

us-central1-docker.pkg.dev/automl-demo-198411/repo-flask/container_flask:latest
[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 130B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/ubuntu:16.04            0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 130B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B      

In [3]:
!gcloud beta artifacts repositories create $ARTIFACT_REPO \
 --repository-format=docker \
 --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [4]:
!docker push $TAG

The push refers to repository [us-central1-docker.pkg.dev/automl-demo-198411/repo-flask/container_flask]

4d12d20c: Preparing 
dd8d4c01: Preparing 
bf18a086: Preparing 
43e66738: Preparing 
83e2cf9e: Preparing 
d2b930fc: Preparing 
ec0db89a: Preparing 
49baa658: Preparing 
4d12d20c: Pushed   136.9MB/129.7MBAPushing  30.71MB/129.7MBlatest: digest: sha256:bbac8a224252151547e513057e555b1e649991e62ab2a21a842f7b4e15ddf60c size: 2199


In [76]:
!gcloud ai-platform models create $MODEL_NAME \
  --region=$REGION \
  --enable-logging \
  --enable-console-logging

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ml engine model [projects/automl-demo-198411/models/model_python].


In [7]:
# Because we're using the custom container we use the beta
!gcloud beta ai-platform versions create $MODEL_VERSION \
  --region=$REGION \
  --model=$MODEL_NAME \
  --machine-type=n1-standard-4 \
  --image=$TAG \
  --ports=5000 \
  --health-route=/ \
  --predict-route=/predict

Using endpoint [https://us-central1-ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [8]:
!gcloud ai-platform versions set-default $MODEL_VERSION \
  --region=$REGION \
  --model=$MODEL_NAME

Using endpoint [https://us-central1-ml.googleapis.com/]
container:
  image: us-central1-docker.pkg.dev/automl-demo-198411/repo-flask/container_flask:latest
  ports:
  - containerPort: 5000
createTime: '2021-01-29T14:38:00Z'
etag: sGPp4ZpdNn4=
isDefault: true
machineType: n1-standard-4
name: projects/automl-demo-198411/models/model_python/versions/v4
routes:
  health: /
  predict: /predict
state: READY


In [9]:
!curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @instances.json \
  https://$REGION-ml.googleapis.com/v1/projects/$PROJECT_ID/models/$MODEL_NAME/versions/$MODEL_VERSION:predict

Prediction result [0.1,0.35,0.4]